In [28]:
from mlflow.tracking import MlflowClient
import mlflow

In [29]:

MLFLOW_TRACKING_URI = "http://localhost:8080"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [30]:
exp_name = "ths-cat-and-dog-experiment"
exp_id = "738720322735546815"
mlflow.set_experiment(exp_name)
client =  MlflowClient()
runs = client.search_runs(experiment_ids=exp_id)
print(len(runs))

1


In [31]:

from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=exp_id,
    filter_string="metrics.val_accuracy > .25",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results = 3,
order_by = ['metrics.val_acc ASC']
)

In [32]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['val_accuracy']:.4f}")

run id: 9b86e9698949428a8e8188e80771e934, rmse: 0.4597


### Register the model

In [33]:
import mlflow
model_uri = 'runs:/9b86e9698949428a8e8188e80771e934/tf_bcc_model'
model_name = "tf_bcc_model"
model_version = mlflow.register_model(model_uri, model_name)
print("model_version : ",model_version)


Registered model 'tf_bcc_model' already exists. Creating a new version of this model...
2024/09/22 03:45:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tf_bcc_model, version 3


model_version :  <ModelVersion: aliases=[], creation_timestamp=1726951541147, current_stage='None', description='', last_updated_timestamp=1726951541147, name='tf_bcc_model', run_id='9b86e9698949428a8e8188e80771e934', run_link='', source='mlflow-artifacts:/738720322735546815/9b86e9698949428a8e8188e80771e934/artifacts/tf_bcc_model', status='READY', status_message='', tags={}, user_id='', version='3'>


Created version '3' of model 'tf_bcc_model'.


In [34]:

client = MlflowClient()
client.set_registered_model_alias(name=model_name,
                                       version="1", alias="production")

client.set_registered_model_alias(name=model_name,
                                       version="2", alias="prod_dep")

### Load model by tag

In [1]:
import mlflow
MLFLOW_TRACKING_URI = "http://localhost:8080"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [2]:

model_name = "tf_bcc_model"
logged_model_uri = f'models:/{model_name}@production'
print(logged_model_uri)

# Load model as a PyFuncModel.
model_requirement_txt = mlflow.pyfunc.get_model_dependencies(logged_model_uri)
print(model_requirement_txt)

/opt/anaconda3/envs/dev_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models:/tf_bcc_model@production


2024/09/22 03:52:17 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/tmpeydnuruz/requirements.txt'.


/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/tmpeydnuruz/requirements.txt


In [3]:
loaded_model = mlflow.pyfunc.load_model(logged_model_uri)

In [4]:
type(loaded_model)

mlflow.pyfunc.PyFuncModel

In [5]:
import tensorflow as tf

img_size = 128
batch_size = 16

root_path = r"/Users/tharhtet/Documents/github/ML-in-Prod-batch-1/5_DeepLearning/tf_best_practices/cats_and_dogs_filtered"
tf_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rescale= 1/255.0

)


test_gen = tf_generator.flow_from_directory(
    root_path+"/test",
    target_size=(img_size, img_size),
    class_mode='binary', #'categorical',
    batch_size=batch_size,
    shuffle=True,
)


#By calling .repeat(), the dataset will loop indefinitely, providing data for all the epochs.

Found 248 images belonging to 2 classes.


In [6]:
test_img = None
test_label = None
for image,label in test_gen:
    print(len(image))
    test_img = image
    test_label = label
    break

16


In [7]:
print(test_img.shape)

(16, 128, 128, 3)


In [8]:

y_pred = loaded_model.predict(test_img)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
[[0.51818943]
 [0.5181898 ]
 [0.51818955]
 [0.51818967]
 [0.51818955]
 [0.51818955]
 [0.51818955]
 [0.51818955]
 [0.51818955]
 [0.5181919 ]
 [0.5181898 ]
 [0.51818955]
 [0.51818955]
 [0.51818955]
 [0.51818955]
 [0.51819015]]
